<a href="https://colab.research.google.com/github/nlp2bok/fintech_/blob/lmh_dev/Data_Preprocessing_bond.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Data Preprocessing

##. 1. 채권분석 보고서(Bond Analysis Report)

1. 채권보고서 파일명(cName_rDate.txt)

 1495개의 채권보고서를 하나의 csv파일에 넣는다.
  파일명에서 cName, rDate추출, 파일 read를 통한 내용 추출('회사명','작성일자','내용')
2. 다양한 Tokenized 방식을 통해 전처리를 실행한다.

 Komoran, Hannanum,  Okt, Kkma, Mecab => Mecab사용
 
 -> Mecab install 필요
3. Token에 한글 태그를 부착한다.
4. 불용어 처리를 실행한다.
5. csv파일에 preprocessing data 컬럼을 추가한다(preData)





In [0]:
import pandas as pd
import numpy as np
import os
import csv

##Step 1. 전체 파일을 하나의 파일로 합치기

In [0]:
#print(dirpath) #현재경로 출력
workDIr = os.path.abspath('/Users/minhyeon/Desktop/BondReport_Text/bText/')
cnt = 0

def fileopen(path):
    f = open(path,'r', encoding='utf-8')
    line = f.read()
    f.close()
    
    return line 

with open('BondFile.csv', 'w', encoding='utf-8', newline='') as cvsFile :
    wr = csv.writer(cvsFile)

    for dirpath, dirnames, filenames in os.walk(workDIr):
        for filename in filenames:
            #파일 컨텐츠 읽기
            path = dirpath+'/'+filename
            fType = filename[-4:]
            try:
                if fType == ".txt" : 
                    text = fileopen(path)
                else :
                    continue
                #파일 타입 체크
            except Exception as ex:
                print(filename + '파일 read 에러가발생 했습니다', ex)


            try:
                rDate = "20" + filename[-12:-4].replace('.','') #Report Date, String형
                cName = filename[:-13] #Company Name
                #print(cnt)
                #print(cName)
                cnt += 1
                wr.writerow([cName,rDate,text])
            except Exception as ex:
                print(filename + '에러가 발생 했습니다', ex)
    
cvsFile.close()

## Step2. Tokenized 실행

##2-1. 파일 읽어오기

In [0]:
#csv파일을 읽어오고 column명을 설정해준다.
mydata = pd.read_csv("BondFile.csv",names=['cName','rDate','bContent'],encoding="utf-8")

In [0]:
#읽어온 데이터를 DataFrame형식으로 담는다.
df = pd.DataFrame(mydata)

In [0]:
#323, 410, 870 error
#mycontent에 전처리할 데이터만 담는다.
mycontent = df['bContent']
print(len(mycontent))

1494


## 2-2. 다양한 방식의 Tokenized 활용해보기


기사별 Tokenized한 값을 komoran_tokens, hannanum_tokens, okt_tokens, kkma_tokens, Mecab_tokens로 각각 저장한 후 결과를 확인해본다.

In [0]:
#출력을 위해서 임시 class 
class List(list): 
    def __str__(self): 
        return "[" + ", ".join(["%s" % x for x in self]) + "]"

In [0]:
#코모란(Komoran) 토큰화
from konlpy.tag import Komoran
komoran = Komoran()
komoran_tokens = []
for i in range(len(mycontent)):
    try:
        komoran_tokens.append(komoran.morphs(mycontent[i]))
        print(i)
    except:
        #이상 데이터 제외
        print(i)
        

In [0]:
#한나눔(Hannanum) 토큰화
from konlpy.tag import Hannanum
hannanum = Hannanum()
hannanum_tokens = []
for i in range(len(mycontent)):
    try:
        hannanum_tokens.append(hannanum.morphs(mycontent[i]))
    except:
        #이상 데이터 제외
        print(i)

In [0]:
#Okt 토큰화
from konlpy.tag import Okt
okt =Okt()
okt_tokens = []
for i in range(len(mycontent)):
    try:
        okt_tokens.append(okt.morphs(mycontent[i]))
    except:
        #이상 데이터 제외
        print(i)

In [0]:
#Kkma 토큰화
from konlpy.tag import Kkma
kkma = Kkma()
kkma_tokens = []
for i in range(len(mycontent)):
    try:
        kkma_tokens.append(kkma.morphs(mycontent[i]))
    except:
        #이상 데이터 제외
        print(i)

In [0]:
#MeCab 토큰화
from ekonlpy.tag import Mecab
mecab = Mecab()
mecab_tokens = []
for i in range(len(mycontent)):
    try:
        #print(mecab.pos(mycontent[i]))
        #break
        mecab_tokens.append(mecab.morphs(mycontent[i]))
        #print(i)
    except:
        #이상 데이터 제외
        print(str(i)+"error")

In [0]:
#토큰화된 데이터 중 첫번째 데이터 확인
print(mecab_tokens[0])

##Step3. Mecab을 사용하여 한글 태그 부착
가장 성능이 좋은 Mecab을 활용하기로 하였음.

한글 태그 부착은 .pos를 사용

In [0]:
#MeCab 토큰화(ekonlpy를 통한 한글 태그 부착 -> ekonlpy 설치 필요)
from ekonlpy.tag import Mecab
mecab = Mecab()
mecab_tokens = []
for i in range(len(mycontent)):
    try:
        #저작권삭제
        mecab_tokens.append(mecab.pos(mycontent[i]))
        #print(i)
    except:
        #이상 데이터 제외
        print(str(i)+'error')
        #이상한 데이터에는 빈 값을 넣어준다.
        mecab_tokens.append(' ')

323error
410error
870error


In [0]:
print(mecab_tokens[323])
#채권보고서1 데이터 토큰화
#print(mecab_tokens[870])

In [0]:
#한글태그 부착확인
#print(mecab_tokens[0][1])
#mecab_tokens[0]

In [0]:
#데이터 개수 확인
print(len(mycontent))
print(len(mecab_tokens))

1494
1494


##Step4. 불용어 처리

(1) stopPos에서는 불필요한 형태소를 한번에 제거

(2) stopWord는 불필요한 단어를 수동으로 제거

mecab-ko-dic 품사태그

명사(NNG), 형용사(VA,VAX), 부사(MAG), 동사(VA)만 추출


In [0]:
##명사(NNG), 형용사(VA,VAX), 부사(MAG), 동사(VA)
includePos = ['NNG','VA','VAX','MAG','VA']
stopWord = ["Microsoft"]
result =[]
preresult = []
cnt = 0
#1493개의 mecab_tokens loop
for mecab_token in mecab_tokens :
    #print(mecab_token)
    #mecab_token(i)번째의 (word/형태소) 데이터에 접근 => wordData = ('단어','형태소')
    cnt+=1
    #print(cnt)
    for wordData in mecab_token:
        #('단어','형태소')중 '형태소' 데이터가 stopPos, '단어' 데이터가 stopWord에 포함되지 않으면 저장한다.
        try:
            if wordData[1] in includePos:
                if wordData[0] not in stopWord:
                    preresult.append(wordData[0])
        except:
            continue
    result.append(','.join(preresult))
    preresult = []

##Step5. 전처리된 데이터를 기존 파일에 추가한다.

In [0]:
#Question3. 데이터 넣는방법 질문...
#print(len(result))
df['preData'] = result
print(df['preData'])

In [0]:

df.to_csv("/Users/minhyeon/Desktop/BondReport_Text/bond_final.csv", header=True, index=False)

In [0]:
text = pd.read_csv("bond_final.csv",encoding="utf-8")